In [1]:
import re

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from pyquery import PyQuery as pq
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
driver = webdriver.Chrome(options=chrome_options)
print(driver.title)

张越博士


In [2]:
def refresh_Pq():
    return pq(driver.page_source)

In [67]:
pi_url_list=[]
l=[]

In [73]:
doc=refresh_Pq()
print(doc('body > div > div.main > div.content.clear > div > div > div.faculty_content span.active').text())
l.append(doc('body > div > div.main > div.content.clear > div > div > div.faculty_content span.active').text())
for i,pi in enumerate(doc.find('body > div > div.main > div.content.clear > div > div > div.faculty_content > div.con_every').items()):
    url=pi('div > a')
    url_text=url.attr('href')
    pi_url_list.append(url_text)
    # print(pi.text())

6


In [75]:
with open('result/xihu_pi_info_url_engineering.txt','a') as f:
    f.write('\n'.join(pi_url_list))

In [76]:
print(len(pi_url_list))

60


## 读取pi personal information

In [3]:
def extract_one_pate_pi_info(pi_link):
    persoanl_page = pq(pi_link, encoding='utf-8')
    chinese_name = persoanl_page(
        'body > div > div.main > div > div.intro.mobile > div.basic_info > p:nth-child(1)').text()
    english_name = persoanl_page(
        'body > div > div.main > div > div.intro.mobile > div.basic_info > p:nth-child(2)').text()[:-7]
    school = persoanl_page('body > div > div.main > div > div.intro.mobile > div.basic_info > p.school')
    school = re.search('school1="(.*?)"', school.text()).group(1)
    website = persoanl_page(
        'body > div > div.main > div > div.intro.mobile > div.basic_info > div > p:nth-child(3) a').text()
    resume = persoanl_page('body > div > div.main > div > div.main_intro > div > div').text()
    return chinese_name,english_name,school,website,resume

In [8]:
from tqdm import tqdm, trange
pi_dic={}
file_list=['result/xihu_pi_info_url_engineering.txt','result/xihu_pi_info_url_science.txt','result/xihu_pi_info_url_SLS.txt',]
for pi_file in file_list:
    print(pi_file)
    with open(pi_file) as f:
        links=f.readlines()
        for pi_link in tqdm(links):
            chinese_name,english_name,school,website,resume=extract_one_pate_pi_info(pi_link.strip())
            pi_dic[chinese_name]=[english_name,school,website,resume]




result/xihu_pi_info_url_engineering.txt


100%|██████████| 60/60 [00:02<00:00, 23.70it/s]


result/xihu_pi_info_url_science.txt


100%|██████████| 55/55 [00:02<00:00, 22.69it/s]


result/xihu_pi_info_url_SLS.txt


100%|██████████| 81/81 [00:03<00:00, 23.16it/s]


In [17]:
westlake_pi_csv_table=pd.DataFrame.from_dict(pi_dic,).T
westlake_pi_csv_table.columns=['english_name','school','website','resume']
westlake_pi_csv_table.index.name='Chinese name'
westlake_pi_csv_table

,english_name,school,website,resume
Chinese name,,,,
陈子博博士,Zibo Chen,生命科学学院与工学院（兼聘）,https://chenlab.org/,个人简介\n\n\n陈子博于2013年在新加坡国立大学获得生命科学一等荣誉学士学位。2013...
程建军博士,Jianjun Cheng,工学院与生命科学学院（兼聘）与理学院（兼聘）,https://www.cheng-lab.com/,个人简介\n\n\n\n程建军教授，1993 年获天津南开大学化学学士学位，1996 年获美...
崔维成博士,Weicheng Cui,工学院,,个人简介\n\n崔维成，1963年生，江苏海门人，“全国优秀科技工作者”（2010）。201...
范迪夏博士,Dixia Fan,工学院,https://www.i4fsi.com/,个人简介\n\n\n范迪夏，1990年出生于上海。2013年获得上海交通大学船舶海洋与建筑工...
郭成辰博士,Chengchen Guo,工学院,https://cguosci.com,个人简介\n\n郭成辰，江苏常州人。2011年6月本科毕业于南京大学化学化工学院，获理学学士...
...,...,...,...,...
郑钜圣博士,"Ju-Sheng Zheng,",生命科学学院,http://zheng.lab.westlake.edu.cn/,个人简介\n\n\n郑钜圣，浙江天台人。2009年毕业于宁波大学，获得学士学位；2014年毕...
周强博士,"Qiang Zhou,",生命科学学院,,个人简介\n周强（1982- ），黑龙江齐齐哈尔人。2000-2004年本科就读于清华大学生...
周挺博士,Ting Zhou,生命科学学院,https://zhouting.lab.westlake.edu.cn,个人简介\n周挺，2006年本科毕业于中国农业大学，2014年博士毕业于中国科学院生物物理研...


## save the pi info

In [18]:
westlake_pi_csv_table.to_csv('result/westlake_pi_csv_table.csv',index=True,encoding='utf8')

In [58]:
for i in doc('img[data-formula]').items():
    print(i.attr('data-formula'))

\mathbb{R}^2
\mathbb{R}^3
\mathbb{R}^n
X
\mathbb{R}
d(x,y)
d(x,y)\geq0
x=y
d(x,y)=d(y,x)
\forall z\in X, d(x,y)\leq d(x,z) + d(z,y)
(X,d)
X
\mathbb{K}
X
\left \| \cdot \right \| :X\rightarrow \mathbb{R}
\left \| x \right \| \geq0
x=0 
\forall \alpha\in \mathbb{K}, \left \| \alpha x \right \|=|\alpha|\left \| x \right \|
\left \| x+y \right \|\leq\left \| x \right \|+\left \| y \right \|
(X,\left \| \cdot \right \|)
X
\mathbb{K}
X
(x,y)
(x, x)\geq0
x=0
(x, y)=\overline{(y, x)}
(\alpha x+\beta y, z)=\alpha(x,z)+\beta(y,z)
x\cdot y
\subset
\subset
\subset
\left \| x \right \|=\sqrt{x\cdot x}=\sqrt{x^2}
d(x,y)=\left \| x-y \right \|=\sqrt{(x-y)\cdot (x-y)}
F:X \rightarrow Y, \forall x,y\in X,a\in\mathbb{R}
f:X\rightarrow Y,\text{satisfies }f-f(0)\text{ is linear}
x=(x_1, x_2, ...,x_n)^T
F:\mathbb{R}^n\rightarrow \mathbb{R}^n
d(F(x),F(y))=d(x,y),\forall x,y \in \mathbb{R}^n
\mathbb{R}^n
F(x)=Ax+b
A
b
\mathbb{R}^n
F(x)=(F_1(x),F_2(x),...,F_n(x))
y=0
 d(F(x),F(0))=d(x,0)\Rightarrow\sum _iF^2_

In [59]:
doc('p img[data-formula]')['data-formula']


TypeError: list indices must be integers or slices, not str

In [26]:
def add_equation(s):
    return '${}$'.format(s)


def add_head(s):
    return '\section'+'{'+s+'}'

def add_figure(url):
    name=url.split('/')[-1].split('.')[0]
    download_image(url=url,name=name,file_path='figure')
    s=r'''\begin{{figure}}[ht]
    \centering
      \includegraphics[width=\textwidth]{{{name}}}
      \caption{{老兄赶快改图片备注}}
    \end{{figure}}'''.format(name=url.split('/')[-1])
    return s

In [ ]:
latex_str = []
for i in doc.children().items():
    if i.is_('p'):
        #公式
        if i.is_('p img[data-formula]'):
            latex_str.append(i.text() + add_equation(i('img').attr('data-formula')))
        else:
            latex_str.append(i.text())
    elif i.is_('h2'):
        latex_str.append(add_head(i.text()))
    elif i.is_('figure'):
        url=i('img').attr('src')
        latex_str.append(add_figure(url))

In [139]:
latex_str

['公式这个算是本科微分几何课程的笔记整理，主要涉及 中的的曲线和 中的曲面，主要参考GTM51 Klingenberg的A Course in Differential Geometry.$\\mathbb{R}^2$',
 '说到这我想起来，刚开学时向修过的同学要实体书，自己打的是Kallenberg，闹了笑话。原因是记得大二时去某校的暑期学校，概率论讨论班用书是他的Fundations of Modern Probability, 读的痛不欲生，从此断了学习概率的念想。',
 '这个笔记应该是比较平和的，如果发现了错误，恳请指出。',
 '(Ps. 本篇笔记前半部分到定理 1.1 是应该知道的预备知识，之后部分是个练习，与主题内容无关，可以略过。)',
 '\\section{1.1 欧氏空间 和欧氏空间中的等距变换(isometry)}',
 '首先定义下度量，范数，内积：',
 '',
 '公式度量：一个从集合 到 的二元函数 ，满足：$X$',
 '公式1.正定性： , 等号成立当且仅当 ;$d(x,y)\\geq0$',
 '公式2.对称性： ;$d(x,y)=d(y,x)$',
 '公式3.三角不等式: ;$\\forall z\\in X, d(x,y)\\leq d(x,z) + d(z,y)$',
 '公式这样就在集合上定义了度量，记作 ，时常略作 ，称之为一个度量空间。$(X,d)$',
 '',
 '公式范数（线性）：一个域 (实数域或者复数域)上的线性空间 ，给任意点定义到原点的距离 ，满足：$\\mathbb{K}$',
 '公式1.正定性： ，等号成立当且仅当 ;$\\left \\| x \\right \\| \\geq0$',
 '公式2.齐次性： ;$\\forall \\alpha\\in \\mathbb{K}, \\left \\| \\alpha x \\right \\|=|\\alpha|\\left \\| x \\right \\|$',
 '公式3.三角不等式： .$\\left \\| x+y \\right \\|\\leq\\left \\| x \\right \\|+\\left \\| y \\right \\|$',
 '公式从而在集合上定义了范数，记作 ，为一个赋范线性空间，时常略作 .$(X

In [147]:
with open('test1.tex','a',encoding='utf8') as f:
    f.write('\n\n'.join(latex_str))

In [30]:
doc=pq(driver.page_source)
p=doc('#root > div > main > div > article > div.Post-RichTextContainer > div > p:nth-child(6)')

In [12]:
p

[<p>]

In [22]:
a=p.pi_link()

In [36]:
p=doc('#root > div > main > div > article > div.Post-RichTextContainer > div > p:nth-child(2)')
p

[<p>]

In [33]:
def add_sring_equation_mix(i):
    a = i.pi_link()
    string_mix=[]
    import re
    regex = re.compile(r'<img.*?>')
    strings = regex.split(a)
    equtions = []
    for e in i('img[data-formula]').items():
        equtions.append(add_equation(e.attr('data-formula')))
    for j in range(len(strings)):
        string_mix.append(strings[j])
        if j<len(equtions):
            string_mix.append(equtions[j])
    return ''.join(string_mix)
add_sring_equation_mix(p)

'GTM51对入门者会难一些，因为直接从最一般的 $n$ 维情况入手，再回头看二三维空间中的曲线，相比之下 <i>Calculus and Analysis in Euclidean Space</i> 这本UTM的Chapter 8 Parametrized Curves 算是相当新手友好的入门内容，由二三维曲线推广到一般情况。</p><p>借用宋老师的话，微分几何的本质是几何，主要通过微积分的手段，用数和函数来刻画几何图形的性质。可能在学习的时候会觉得明明很直观，几何画出来就是这样的，但是要用规范的严格语言说出来很困难。所以说也是一个打基础的课程，先学会怎样严格的表述，在这基础之上才能正确的使用微积分这个工具。      </p><h2>2.1 参数曲线(Parametrized Curves)</h2><p><b>Definition 2.1.1</b>\u2002 参数曲线 是指一个光滑映射(smooth mapping)：</p><p> </p><p>这里  ,  且非空。 更进一步，如果  ，称这样的曲线是正则的(regular).  </p><p>也就是说我们平时说的“曲线”，其实是一个映射，平时说“点在曲线上”，是指这一点，如  在这个映射的像集中。</p><p>我们一般都要求曲线是正则的，因为是光滑映射，所以导数不等于零  导数的范数不等于零，直观理解是这样的曲线是不会停下的，也因此我们可以用隐函数定理，反函数定理等。</p><p>如果  是不是开区间，定义的意思是，存在一个包含  的开区间  <i>，使得</i>  在  <i>上是一个光滑映射，并且</i>  ，这样就解决了端点无定义的问题。</p><p>这里我们说的参数曲线都是“光滑”的，这个要求也太高了，不过现在考虑的都是很显然，一眼能够看明白的曲线，以后会逐步放宽。    </p><p><b>Definition 2.1.2</b>   i)切空间(Tangent Space): 对  ，切空间是指所有以  为起点的  维向量所构成的空间，记作  .</p><p>ii)沿着曲线c的向量场(Vector field along  ): 指一个可微映射  ，  ，  .</p><p>iii)切向量场(Tangent vector field of   ): 指一个沿着 c 的向量场，其中在  

In [35]:
p.text()

'GTM51对入门者会难一些，因为直接从最一般的 维情况入手，再回头看二三维空间中的曲线，相比之下 Calculus and Analysis in Euclidean Space 这本UTM的Chapter 8 Parametrized Curves 算是相当新手友好的入门内容，由二三维曲线推广到一般情况。'